<a href="https://colab.research.google.com/github/thomas0913/BlockChain_BasicCase/blob/main/BlockChain_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 定義區塊鏈格式與架構

*   交易格式

In [ ]:
class Transaction:
  def __init__(self,sender,receiver,amounts,fee,message):
    self.sender = sender
    self.receiver = receiver
    self.amounts = amounts
    self.fee = fee
    self.message = message

*   區塊格式

